### import packages

In [1]:
import scanpy as sc 
from SpaJoint.SpaJoint import SpaJoint
import pandas as pd
import numpy as np

c:\Users\18813\anaconda3\envs\SpaJoint\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\18813\anaconda3\envs\SpaJoint\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\18813\anaconda3\envs\SpaJoint\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\18813\anaconda3\envs\SpaJoint\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### import data

In [2]:
dataset = "example"
ref_adata = sc.read("./" + dataset + "/scRNA.h5ad")
inf_adata = sc.read("./" + dataset + "/ST.h5ad")
save_dir = "./" + dataset
print(ref_adata.shape)
print(inf_adata.shape)

(512, 313)
(512, 313)


### Add spatial coordinates to ST data

In [3]:
inf_adata.obsm['spatial'] = np.array(inf_adata.obs[['x','y']])

### Convert cell type labels to numeric codes

In [4]:
ref_adata.obs["CellType"] = ref_adata.obs["cell_type"].copy()
ref_adata.obs["CellType"] = ref_adata.obs["CellType"].astype("category")
ref_adata.obs["celltype"]=ref_adata.obs["CellType"].astype("category").cat.codes 

### Run the algorithm

In [5]:
spajoint = SpaJoint(ref_ds=ref_adata,inf_ds=inf_adata,save_dir=save_dir)
spajoint.PrepareDataLoader_stage()
spajoint.BuildNet_stage()
spajoint.TrainingProcessStage()

c:\Users\18813\anaconda3\envs\SpaJoint\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
c:\Users\18813\anaconda3\envs\SpaJoint\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
d:\scJoint_loc\code_github\SpaJoint\utils.py:215: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns["overlap_genes"] = genes
d:\scJoint_loc\code_github\SpaJoint\utils.py:216: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata_sc.uns["overlap_genes"] = genes
2025-04-16 13:20:20,438 - root - INFO - Create log file....
2025-04-16 13:20:20,440 - root - INFO - Create SpaJoint Object Done....


Number of overlap genes: 234
num_workers: 0
Epoch: 0
rna_reduction_loss = 2.6297826766967773,st_reduction_loss=2.6203908920288086,sim_loss=0.5593385100364685,loc_loss=4.063313868097994.
encoder_loss = 2.2758472495183417,cell_loss=2.3323464393615723.
rna_reduction_loss = 2.618713855743408,st_reduction_loss=2.6380810737609863,sim_loss=0.5881190299987793,loc_loss=4.095367777983083.
encoder_loss = 2.278488276131895,cell_loss=2.161231517791748.
Epoch: 1
rna_reduction_loss = 2.673264265060425,st_reduction_loss=2.6216447353363037,sim_loss=0.5456951856613159,loc_loss=4.107033282836114.
encoder_loss = 2.30074041492141,cell_loss=1.9669524431228638.
rna_reduction_loss = 2.652665376663208,st_reduction_loss=2.623849391937256,sim_loss=0.5434529781341553,loc_loss=4.096563851974205.
encoder_loss = 2.2935766768737227,cell_loss=1.7443439960479736.
Epoch: 2
rna_reduction_loss = 2.61285138130188,st_reduction_loss=2.6118741035461426,sim_loss=0.5515642166137695,loc_loss=4.094197528383365.
encoder_loss = 2.2

In [6]:
celltype_density = pd.DataFrame(spajoint.inf_ds.obsm["cell_pred"])
cell_type_dict = dict(zip(ref_adata.obs['celltype'],ref_adata.obs['CellType']))
celltype_density.rename(columns=cell_type_dict,inplace=True)
celltype_density.index = inf_adata.obs.index
celltype_density.to_csv(save_dir +'/spajoint_result.txt')

### deconvolution result with dimension spot * cell type

In [7]:
celltype_density.head()

,B-cells,CAFs,Endothelial,Myeloid,Normal Epithelial,PVL,T-cells
AGTAACTATAGCAGCC-1,0.114704,0.047078,0.432493,0.129550,0.125670,0.079525,0.070980
TGGGCGATACAATAAG-1,0.062942,0.210621,0.074113,0.345572,0.182475,0.070816,0.053461
ATCCTTCTGAAAGAAC-1,0.122005,0.010643,0.114129,0.448212,0.193979,0.049621,0.061411
TCGATGTTACGGCCGT-1,0.028108,0.041824,0.071938,0.017568,0.760767,0.064658,0.015137
AACCCAGAGACGGAGA-1,0.007317,0.008383,0.007714,0.010524,0.956626,0.007861,0.001575
